In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException, WebDriverException 
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import psutil

In [3]:
dh = pd.read_csv("path2project/Annuaire/AnnuaireFiltre.csv", encoding='ANSI')

In [4]:
dh = dh[dh['etablissementSiege'] == True]

In [42]:
#dh.to_csv('path2project/Annuaire/AnnuaireFinal.csv', index = False,  encoding='ANSI')

In [5]:
di = pd.concat([dh.iloc[:,0], dh.iloc[:,2:6], dh.iloc[:,7:10], dh.iloc[:,23:29]],axis=1)

In [6]:
def sigleMaker(string):
    articles = ['D\'', 'DE', 'DES', 'DU', 'ET','LE','LA']
    words = str(string).split()
    for l in words:
        if l in articles:
                words.remove(l)
    sigle = ''.join(word[0] for word in words)
    print(sigle)
    return(sigle)

In [ ]:
Search = []
for i in range(len(di)):
     Zip = zip(di.columns, di.iloc[i])
     sigle = sigleMaker(di["denominationUniteLegale"].iloc[i])
     adresse = ''
     for j, k in Zip:
          if di.columns.get_loc(j) in range(0, 8) and not pd.isna(k) and k != sigle:
                    nomEtablissement = str(k)
          elif di.columns.get_loc(j) >= 8 and not pd.isna(k) and k!=0:
               adresse = ' '.join([adresse, str(k)])
     search = ' '.join([nomEtablissement, adresse])
     Search.append(search)

print(Search)

In [2]:
def Driver():    
    driver = webdriver.Chrome('chromedriver')

    #This line creates a ChromeOptions object, which allows you to customize the behavior of the ChromeDriver.
    options=webdriver.ChromeOptions()

    #window size optimised for easier element selection
    options.add_argument("--window-size=600x1080") 

    #This is useful for running automated tests or performing web scraping tasks without displaying the browser window.
    options.add_argument('-headless')

    #disable chrome contenairiation in order to modify options
    options.add_argument('-no-sandbox')

    #disable shared memory space in Chrome cause might be troublesome in headless mode
    options.add_argument('-disable-dev-shm-usage')

    #ignores SSL certificate errors
    options.add_argument('--ignore-certificate-errors')

    #pass in incognito mode
    options.add_argument("incognito")

    #set chrome in french
    options.add_argument('--lang=fr')
    
    #pass arguments
    driver = webdriver.Chrome('chromedriver',options=options)

    return driver

In [3]:
def go2maps(driver):
    #Print the new window size
    print(driver.get_window_rect())
    #drive to website
    driver.get(websiteUrl)
    #sleep
    time.sleep(5)
    #refuse cookies
    RejectAll = driver.find_element(By.TAG_NAME, "button")
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.TAG_NAME, "button"))
    )
    RejectAll.click()
                             

In [4]:
def mapsSearch(driver, search):
    searchboxSelector = '#searchboxinput'
    #wait for maps searchbox to be clickable
    WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, searchboxSelector))
     )
    searchbox = driver.find_element(By.CSS_SELECTOR,searchboxSelector)
    searchbox.clear()
    #input search pattern to maps searchbox
    searchbox.send_keys(search)
    searchbox.send_keys(Keys.ENTER)
    

In [5]:
def bytes_to_gb(bytes_value):
    gb_value = bytes_value / (1024**3)
    return gb_value

def bytes_to_mb(bytes_value):
    gb_value = bytes_value / (1024**2)
    return gb_value

def get_current_memory_usage():
    # Get the current process ID
    current_process = psutil.Process()

    # Get memory usage information
    memory_info = current_process.memory_info()

    # Convert the memory usage from bytes to megabytes
    memory_usage_mb = bytes_to_mb(memory_info.rss)

    # Convert the memory usage from bytes to gigabytes
    memory_usage_gb = bytes_to_gb(memory_info.rss)

    return memory_usage_mb, memory_usage_gb



In [31]:
adressesEcoles = pd.read_csv('path2project/Annuaire/adressesEcoles')

In [32]:
adressesEcoles 

,AdresseEtablissementPrive,NomEtablissementPrive
0,1 AVENUE RATISBONNE 91000 EVRY-COURCOURONNES,"['Lycee polyvalent Notre-Dame de Sion', 'Colle..."
1,1 AVENUE DE VERDUN 95300 PONTOISE,"['College Saint-Martin de France', 'College Pr..."
2,1 AVENUE DE LA BRETECHE 78150 LE CHESNAY-ROCQU...,"['College Blanche de Castille', 'Lycee Blanche..."
3,1 BOULEVARD DU NORD 93340 LE RAINCY,"['College Tebrotzassere', 'College Tebrotzasse..."
4,1 DOMAINE DU VIGNOIS 77440 ARMENTIERES-EN-BRIE,"['Lycee general prive Beth Myriam', 'College p..."
...,...,...
1904,RUE DU 8 MAI 1945 91120 PALAISEAU,['Lycee Polyvalent Prive Saint Martin']
1905,SITE DU BEL-AIR 51 BOULEVARD DE LA PAIX 78100 ...,['Ecole Sup de V - CCI Paris Ile-de-France']
1906,"VOG ACADEMY, HOTEL MARIOTT 17 BOULEVARD SAINT-...","[""Ecole de Coiffure Formul'Academie""]"
1907,ZA 15 RUE DE LA MARE BLANCHE 77186 NOISIEL,['Institut Superieur du Transport et de la Log...


In [33]:
Na = ['NA'] * len(adressesEcoles)

In [34]:
mapsSchools = pd.DataFrame({
    'mapsName': Na,
    'mapsAddresses': Na,    
    'mapsPhone':  Na,
    'mapsWebsite': Na
})

In [35]:
adressesEcoles = pd.concat([adressesEcoles, mapsSchools], axis = 1 )

Must set multiple Index in merging intead of grouping!!!!!!!!!!!!!!!!!

In [38]:
adressesEcoles.set_index(['AdresseEtablissementPrive', 'NomEtablissementPrive'])

,,mapsName,mapsAddresses,mapsPhone,mapsWebsite
AdresseEtablissementPrive,NomEtablissementPrive,,,,
1 AVENUE RATISBONNE 91000 EVRY-COURCOURONNES,"['Lycee polyvalent Notre-Dame de Sion', 'College Prive Notre Dame de Sion']",NA,NA,NA,NA
1 AVENUE DE VERDUN 95300 PONTOISE,"['College Saint-Martin de France', 'College Prive Saint-Martin-de-France', 'Lycee Prive Saint-Martin-de-France']",NA,NA,NA,NA
1 AVENUE DE LA BRETECHE 78150 LE CHESNAY-ROCQUENCOURT,"['College Blanche de Castille', 'Lycee Blanche de Castille', 'Lycee Prive Blanche-de-Castille', 'College Prive Blanche-de-Castille', ""Lycee d'Enseignement General Prive Blanche-de-Castille""]",NA,NA,NA,NA
1 BOULEVARD DU NORD 93340 LE RAINCY,"['College Tebrotzassere', 'College Tebrotzassere']",NA,NA,NA,NA
1 DOMAINE DU VIGNOIS 77440 ARMENTIERES-EN-BRIE,"['Lycee general prive Beth Myriam', 'College prive Beth Myriam', 'Ecole Primaire Privee David Tsemah']",NA,NA,NA,NA
...,...,...,...,...,...
RUE DU 8 MAI 1945 91120 PALAISEAU,['Lycee Polyvalent Prive Saint Martin'],NA,NA,NA,NA
SITE DU BEL-AIR 51 BOULEVARD DE LA PAIX 78100 SAINT-GERMAIN-EN-LAYE,['Ecole Sup de V - CCI Paris Ile-de-France'],NA,NA,NA,NA
"VOG ACADEMY, HOTEL MARIOTT 17 BOULEVARD SAINT-JACQUES 75014 PARIS 14","[""Ecole de Coiffure Formul'Academie""]",NA,NA,NA,NA


In [52]:
def retrieve_mapsElements(driver, data):
    nameSelector = '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.TIHn2 > div > div.lMbq3e > div:nth-child(1)'
    elementsAttributes = '[data-tooltip][data-item-id]'
    #Wait for name element to be located
    WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, nameSelector))
    )     
    #Wait for all elements to be located                             
    WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR,elementsAttributes))
    )  
    elements = driver.find_elements(By.CSS_SELECTOR, value=elementsAttributes) 
    name     = driver.find_element(By.CSS_SELECTOR, nameSelector).text
    data['mapsName'] = name
    for element in elements:
        attribute_pattern = element.get_attribute('data-tooltip')
        if attribute_pattern == 'Copier l\'adresse':
            data['mapsAddress'] = element.text
        elif attribute_pattern == 'Accéder au site Web':
            data['mapsSite'] = element.text
        elif attribute_pattern == 'Copier le numéro de téléphone':
            data['mapsPhone'] = element.text
    return data

In [53]:
d1 = pd.read_csv('path2project/Annuaire/Maps/df4419.csv', encoding = 'utf-16', sep = '\t') 
d2 = pd.read_csv('path2project/Annuaire/Maps/d2.csv', encoding = 'utf-16', sep = '\t')
d3 = pd.read_csv('path2project/Annuaire/Maps/Maps6381.csv', encoding = 'utf-16')    
d4 = pd.read_csv('path2project/Annuaire/Maps/d4.csv', encoding = 'utf-16', sep = '\t')
d5 = pd.read_csv('path2project/Annuaire/Maps/Maps6427.csv', encoding = 'ANSI')

In [54]:
m = pd.concat([d1, d2, d3, d4, d5], axis = 0)

m.replace(['NA', 'NaN'], np.nan)

na = m[m['mapsName'].isna() == True]

#  Mark duplicates as True except for the first occurrence.
replicants = m[m.duplicated(keep ='first')]

replicants = replicants.dropna(subset = ['mapsName'])

In [83]:
len(Search)

6428

In [82]:
len(m)

6431

In [87]:
#len(m) = 6431 != len(Search) = 6428 =>  3 corrupted indexes

6431

In [191]:
m = m.drop('Unnamed: 0', axis = 1)

In [ ]:
adressesEcoles = pd.read_csv('path2project/Annuaire/adressesEcoles')

In [ ]:
#init WebDriver
driver = Driver()
go2maps(driver)
#Init variables
index = list(na.index) + list(replicants.index)
data = {}
MasquerItineraireSelector = '#omnibox-directions > div > div:nth-child(2) > div > button'
ThumbnailsClass = "hfpxzc"
#search pattern for each element in Search list
for i, j in enumerate(index):
    if i!=0 and i%50 == 0:
        driver.quit()
        driver = Driver()
        go2maps(driver)
    if int(i) in range(len(index)):
        data['mapsName']    = 'NA' 
        data['mapsAddress'] = 'NA' 
        data['mapsSite']    = 'NA' 
        data['mapsPhone']   = 'NA' 
        data['firstMatch']  = 'NA'
        time.sleep(10)
        #case number 1: 
            #page had the time to load and we've got a first match
        search = Search[j]
        try:
            mapsSearch(driver, search)
            time.sleep(5)  
            data = retrieve_mapsElements(driver, data)
            data['firstMatch']  = 'True'
        except (TimeoutException, StaleElementReferenceException, NoSuchElementException, WebDriverException):
            #case number 2: 
                #page had the time to load but we've got to choose between multiple thumbnailes (second match)
            try:   
                #drive to firstThumbnail if exists
                    time.sleep(5)
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, ThumbnailsClass))
                    )
                    WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, ThumbnailsClass))
                    )
                    firstThumbnail = driver.find_element(By.CLASS_NAME, ThumbnailsClass)
                    firstThumbnail.click()
                    time.sleep(20)
                    print(driver.current_url)
                    data = retrieve_mapsElements(driver, data)
                    data['firstMatch'] = 'False'
            except (TimeoutException, StaleElementReferenceException, NoSuchElementException, WebDriverException):
                #case number 3:
                    #page didn't has the time to load => maps gets the wrong element (Itineraire)
                try:
                    #therefore a cross allowing to close search of "itineraire" appears
                    time.sleep(5)  
                    WebDriverWait(driver,5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, MasquerItineraireSelector))
                    )
                    MasquerItineraire = driver.find_element(By.CSS_SELECTOR, MasquerItineraireSelector)
                    MasquerItineraire.click()
                    #retry search
                    time.sleep(5)  
                    mapsSearch(driver, search)
                    time.sleep(5)  
                    data = retrieve_mapsElements(driver, data)
                    data['firstMatch']  = 'True'
                #case number 4: 
                    #no match
                except (TimeoutException, StaleElementReferenceException, NoSuchElementException):
                    data['mapsName']    = 'NA' 
                    data['mapsAddress'] = 'NA' 
                    data['mapsSite']    = 'NA' 
                    data['mapsPhone']   = 'NA' 
                    data['firstMatch']  = 'NA' 
                except  WebDriverException:
                    #reinit WebDriver
                    driver.quit()
                    driver = Driver()
                    go2maps(driver)
                    i-=1
        print(i, search, m.iloc[j])
        #data dictionary convertion           
        data = pd.Series(data)
        print(data)
        #data actualisation 
        m.iloc[j] = data
        print(m.iloc[j])

In [167]:
m.drop('Unnamed: 0', axis = 1, inplace = True)

In [169]:
m.to_csv('path2project/Annuaire/Maps/finalMaps.csv', encoding = 'utf-16', sep = '\t', index = False)

In [171]:
m = pd.read_csv('path2project/Annuaire/Maps/finalMaps.csv', encoding = 'utf-16', sep = '\t')

In [172]:
len(m)

6428

In [197]:
di.reset_index(inplace = True)

In [204]:
df = pd.concat([di, m], axis = 1)

In [206]:
df.to_csv('path2project/Annuaire/Maps/AnnuaireFinal+Maps.csv', encoding = 'utf-16', sep = '\t', index = False)